# Model Training

In this project, I compared two neural network architectures: a times series CNN and a ResNet for analyzing ECG time series data, with the goal of predicting normal versus abnormal ECGs. The choices I made in selecting and designing these models were informed by my research and experience working in the field of ECG analysis.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import ast
import pickle

import optuna
from tensorflow.keras.callbacks import EarlyStopping

from models.resnet import *
from models.tscnn import *
from utils.data_preprocessing import *
from utils.hyperparameter_tuning import *

pd.set_option("display.max_columns", None)

In [3]:
data_path = "../data/"
sampling_rate = 100
input_shape = (1000, 12)
num_classes = 1
batch_size = 32
epochs = 50

In [4]:
Y = pd.read_csv(join(data_path, "processed/labels.csv"))

For initial experiments, I use 50% of the data to accelerate model training given my limited access to computing resources. Models are trained on a CPU.

I follow the train/validation/test splitting guidelines from the PTB-XL documentation. This is roughly equivalent to a 80-10-10 split. These folds ensure that records from the same patient remain within the same fold. 

In [5]:
window_size = 100
step_size = 100
sample_size = 0.5

train_generator, val_generator, test_generator = create_generators_or_datasets(
    y=Y,
    window_size=window_size,
    step_size=step_size,
    batch_size=batch_size,
    sample_size=sample_size,
    sampling_rate=sampling_rate,
    data_path=data_path,
    return_generators=True,
)

X_train, y_train, X_val, y_val, X_test, y_test = create_generators_or_datasets(
    y=Y,
    window_size=window_size,
    step_size=step_size,
    batch_size=batch_size,
    sample_size=sample_size,
    sampling_rate=sampling_rate,
    data_path=data_path,
    return_generators=False,
)

In [6]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((4901, 1000, 12), (4901,), (604, 1000, 12), (604,), (656, 1000, 12), (656,))

## Times Series Convolutional Neural Network

I chose this CNN model for its efficiency in real-time ECG analysis. I designed the architecture with convolutional and pooling layers to quickly and effectively extract key features from the data. By using a sliding window approach, I can process segments of the ECG signal independently, allowing the model to capture local patterns over time without needing the entire signal at once. I used `Conv1D` layers because they are well-suited for sequential data like ECG signals. 

In [7]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
baseline_cnn = TimeSeriesCNN(
    input_shape=input_shape, window_size=window_size, dropout_rate=0.0, l2_reg=0.0, filters=[64, 128, 256]
)
history = baseline_cnn.train(train_generator, val_generator, epochs=epochs, batch_size=32, callbacks=[early_stopping])

loss, accuracy, auc, predictions = evaluate_cnn_performance(baseline_cnn, val_generator, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 56s 35ms/step - accuracy: 0.7507 - auc: 0.8300 - loss: 0.4920 - val_accuracy: 0.8334 - val_auc: 0.9429 - val_loss: 0.3551
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 58s 38ms/step - accuracy: 0.8704 - auc: 0.9414 - loss: 0.3090 - val_accuracy: 0.8626 - val_auc: 0.9473 - val_loss: 0.3061
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 68s 44ms/step - accuracy: 0.8758 - auc: 0.9467 - loss: 0.2942 - val_accuracy: 0.8515 - val_auc: 0.9428 - val_loss: 0.4016
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 61s 40ms/step - accuracy: 0.8900 - auc: 0.9549 - loss: 0.2735 - val_accuracy: 0.8796 - val_auc: 0.9536 - val_loss: 0.2908
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 52s 34ms/step - accuracy: 0.8952 - auc: 0.9614 - loss: 0.2503 - val_accuracy: 0.8803 - val_auc: 0.9539 - val_loss: 0.2949
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - accuracy: 0.9018 - auc: 0.9642 - loss: 0.2398 - val_accuracy: 0.8810 - val_auc: 0.9552 - val_loss: 0.2853
Epoch 7/50
1532/

### Approaching Overfitting

While the model is performing well, with a validation AUC of 0.964, I have observed significant overfitting, evidenced by a much lower training set loss compared to the validation set loss. The following experiments explore the impact of dropout, L2 regularization, and convolution layer sizes to address this overfitting.

In [8]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
cnn = TimeSeriesCNN(
    input_shape=input_shape, window_size=window_size, dropout_rate=0.4, l2_reg=0.0, filters=[64, 128, 256]
)
history = cnn.train(train_generator, val_generator, epochs=epochs, batch_size=32, callbacks=[early_stopping])

loss, accuracy, auc, predictions = evaluate_cnn_performance(cnn, val_generator, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 139s 86ms/step - accuracy: 0.6986 - auc: 0.7716 - loss: 0.5566 - val_accuracy: 0.8629 - val_auc: 0.9364 - val_loss: 0.3285
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 80s 52ms/step - accuracy: 0.8527 - auc: 0.9245 - loss: 0.3448 - val_accuracy: 0.8558 - val_auc: 0.9418 - val_loss: 0.3332
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 118s 77ms/step - accuracy: 0.8620 - auc: 0.9358 - loss: 0.3250 - val_accuracy: 0.8536 - val_auc: 0.9473 - val_loss: 0.3298
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 117s 76ms/step - accuracy: 0.8756 - auc: 0.9427 - loss: 0.3056 - val_accuracy: 0.8508 - val_auc: 0.9484 - val_loss: 0.3270
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 239s 156ms/step - accuracy: 0.8690 - auc: 0.9398 - loss: 0.3118 - val_accuracy: 0.8742 - val_auc: 0.9482 - val_loss: 0.2862
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 97s 63ms/step - accuracy: 0.8804 - auc: 0.9451 - loss: 0.2997 - val_accuracy: 0.8793 - val_auc: 0.9514 - val_loss: 0.2793
Epoch 7/50


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
cnn = TimeSeriesCNN(
    input_shape=input_shape, window_size=window_size, dropout_rate=0.0, l2_reg=0.001, filters=[64, 128, 256]
)
history = cnn.train(train_generator, val_generator, epochs=epochs, batch_size=32, callbacks=[early_stopping])

loss, accuracy, auc, predictions = evaluate_cnn_performance(cnn, val_generator, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 52s 31ms/step - accuracy: 0.7342 - auc: 0.8129 - loss: 0.6618 - val_accuracy: 0.8472 - val_auc: 0.9325 - val_loss: 0.4155
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 47s 30ms/step - accuracy: 0.8540 - auc: 0.9255 - loss: 0.4151 - val_accuracy: 0.8609 - val_auc: 0.9397 - val_loss: 0.3941
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 48s 31ms/step - accuracy: 0.8539 - auc: 0.9269 - loss: 0.4017 - val_accuracy: 0.8247 - val_auc: 0.9407 - val_loss: 0.4368
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 53s 35ms/step - accuracy: 0.8613 - auc: 0.9320 - loss: 0.3846 - val_accuracy: 0.8654 - val_auc: 0.9432 - val_loss: 0.3568
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 57s 37ms/step - accuracy: 0.8747 - auc: 0.9397 - loss: 0.3650 - val_accuracy: 0.8694 - val_auc: 0.9451 - val_loss: 0.3492
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 54s 35ms/step - accuracy: 0.8770 - auc: 0.9436 - loss: 0.3499 - val_accuracy: 0.8644 - val_auc: 0.9441 - val_loss: 0.3692
Epoch 7/50
1532/

In [10]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
cnn = TimeSeriesCNN(
    input_shape=input_shape, window_size=window_size, dropout_rate=0.0, l2_reg=0.0, filters=[32, 64, 128]
)
history = cnn.train(train_generator, val_generator, epochs=epochs, batch_size=32, callbacks=[early_stopping])

loss, accuracy, auc, predictions = evaluate_cnn_performance(cnn, val_generator, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 24s 14ms/step - accuracy: 0.7509 - auc: 0.8326 - loss: 0.4891 - val_accuracy: 0.8563 - val_auc: 0.9422 - val_loss: 0.3155
Epoch 2/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 23s 15ms/step - accuracy: 0.8661 - auc: 0.9388 - loss: 0.3165 - val_accuracy: 0.8551 - val_auc: 0.9433 - val_loss: 0.3169
Epoch 3/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8782 - auc: 0.9437 - loss: 0.3023 - val_accuracy: 0.8722 - val_auc: 0.9505 - val_loss: 0.2873
Epoch 4/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.8876 - auc: 0.9514 - loss: 0.2768 - val_accuracy: 0.8816 - val_auc: 0.9534 - val_loss: 0.2745
Epoch 5/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8917 - auc: 0.9577 - loss: 0.2587 - val_accuracy: 0.8823 - val_auc: 0.9549 - val_loss: 0.2737
Epoch 6/50
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 30s 19ms/step - accuracy: 0.9002 - auc: 0.9632 - loss: 0.2423 - val_accuracy: 0.8747 - val_auc: 0.9543 - val_loss: 0.3059
Epoch 7/50
1532/

### Hyperparameter tuning

Reducing model complexity through filter sizes does not seem to improve overfitting or model performance. However, while dropout and regularization have a positive impact on overfitting, the model performs worse on the validation set loss than our baseline. I will tune these parameters along with window size and step size using Optuna. Optuna's adaptive search capabilities allow for effective exploration of the hyperparameter space. For efficiency, I use 25% of the data.

In [11]:
# Lambda function to pass additional parameters

study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(
        trial,
        objective_metric="val_loss",
        input_shape=input_shape,
        Y=Y,
        sampling_rate=sampling_rate,
        sample_size=0.25,
        data_path=data_path,
        batch_size=batch_size,
        num_classes=num_classes,
        model_class=TimeSeriesCNN,
        filters=[64, 128, 256],
        epochs=epochs,
    ),
    n_trials=200,
    timeout=60 * 60 * 5,
)


print(f"Best trial: {study.best_trial.value}")
print(f"Best window_size: {study.best_trial.params['window_size']}")
print(f"Best step_size: {study.best_trial.params['step_size']}")
print(f"Best dropout_rate: {study.best_trial.params['dropout_rate']}")
print(f"Best l2_reg: {study.best_trial.params['l2_reg']}")

[I 2024-08-27 10:10:42,173] A new study created in memory with name: no-name-f1999ddd-e8d1-4f17-871f-cf7e9c9304bb
[I 2024-08-27 10:15:44,505] Trial 0 finished with value: 0.3307325839996338 and parameters: {'window_size': 191, 'step_size': 165, 'l2_reg': 9.159969462271967e-09, 'dropout_rate': 0.3496380249038563}. Best is trial 0 with value: 0.3307325839996338.
[I 2024-08-27 10:25:46,619] Trial 1 finished with value: 0.253520667552948 and parameters: {'window_size': 96, 'step_size': 56, 'l2_reg': 5.609971398816462e-07, 'dropout_rate': 0.3284531233399967}. Best is trial 1 with value: 0.253520667552948.
[I 2024-08-27 10:29:38,111] Trial 2 finished with value: 0.27166062593460083 and parameters: {'window_size': 96, 'step_size': 77, 'l2_reg': 4.515053448565958e-09, 'dropout_rate': 0.2956701707108168}. Best is trial 1 with value: 0.253520667552948.
[I 2024-08-27 10:40:15,140] Trial 3 finished with value: 0.27635514736175537 and parameters: {'window_size': 178, 'step_size': 74, 'l2_reg': 1.68

Best trial: 0.24977916479110718
Best window_size: 196
Best step_size: 75
Best dropout_rate: 0.43035840459539587
Best l2_reg: 3.6206124274583214e-11


In [12]:
with open("../reports/tscnn_hyperparameter_study.pkl", "wb") as f:
    pickle.dump(study, f)

### Final Model 
Let's train the optimal model on all of the data.

In [13]:
window_size = study.best_trial.params["window_size"]
step_size = study.best_trial.params["step_size"]
dropout_rate = study.best_trial.params["dropout_rate"]
l2_reg = study.best_trial.params["l2_reg"]

train_generator, val_generator, test_generator = create_generators_or_datasets(
    y=Y,
    window_size=window_size,
    step_size=step_size,
    batch_size=batch_size,
    sample_size=1.0,
    sampling_rate=sampling_rate,
    data_path=data_path,
    return_generators=True,
)

In [14]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
ecg_cnn = TimeSeriesCNN(
    input_shape=input_shape, window_size=window_size, dropout_rate=dropout_rate, l2_reg=l2_reg, filters=[64, 128, 256]
)
history = ecg_cnn.train(
    train_generator, val_generator, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping]
)

loss, accuracy, auc, predictions = evaluate_cnn_performance(ecg_cnn, val_generator, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 170s 49ms/step - accuracy: 0.7686 - auc: 0.8528 - loss: 0.4585 - val_accuracy: 0.8203 - val_auc: 0.9414 - val_loss: 0.4530
Epoch 2/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 192s 56ms/step - accuracy: 0.8664 - auc: 0.9357 - loss: 0.3232 - val_accuracy: 0.8098 - val_auc: 0.9489 - val_loss: 0.4207
Epoch 3/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 274s 80ms/step - accuracy: 0.8766 - auc: 0.9440 - loss: 0.3011 - val_accuracy: 0.8831 - val_auc: 0.9533 - val_loss: 0.2758
Epoch 4/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 283s 83ms/step - accuracy: 0.8853 - auc: 0.9493 - loss: 0.2843 - val_accuracy: 0.8694 - val_auc: 0.9515 - val_loss: 0.3081
Epoch 5/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 260s 76ms/step - accuracy: 0.8882 - auc: 0.9548 - loss: 0.2713 - val_accuracy: 0.8803 - val_auc: 0.9558 - val_loss: 0.2831
Epoch 6/50
3410/3410 ━━━━━━━━━━━━━━━━━━━━ 260s 76ms/step - accuracy: 0.8931 - auc: 0.9557 - loss: 0.2686 - val_accuracy: 0.8861 - val_auc: 0.9572 - val_loss: 0.2703
Epoch 7/50

In [15]:
ecg_cnn.model.save("../models/cnn_model.keras", include_optimizer=False)

with open("../reports/cnn_history.pkl", "wb") as f:
    pickle.dump(history.history, f)

In [16]:
loss, accuracy, auc, predictions = evaluate_cnn_performance(ecg_cnn, test_generator, y_test)
np.save("../results/cnn_preds.npy", predictions)

## ResNet

I chose to compare the CNN with a ResNet model because ResNet's residual connections allow it to capture complex patterns and long-range dependencies in ECG data, which a standard CNN might miss. Unlike the CNN, ResNet mitigates the vanishing gradient problem, making it more effective at learning deep features. Additionally, the global feature aggregation layer in ResNet enhances generalization, improving its ability to consistently detect important patterns across various ECG signals.

In [17]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
resnet = ECGResNet(input_shape=input_shape, num_classes=num_classes, l2_reg=0.0, dropout_rate=0.0)
resnet_history = resnet.train(
    X_train, y_train, X_val, y_val, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping]
)

loss, accuracy, auc, predictions = evaluate_resnet_performance(resnet, X_val, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 141s 860ms/step - accuracy: 0.8095 - auc: 0.8861 - loss: 0.4180 - val_accuracy: 0.5430 - val_auc: 0.8560 - val_loss: 0.8135
Epoch 2/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 125s 813ms/step - accuracy: 0.8747 - auc: 0.9481 - loss: 0.2885 - val_accuracy: 0.8659 - val_auc: 0.9482 - val_loss: 0.3181
Epoch 3/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 137s 893ms/step - accuracy: 0.8842 - auc: 0.9524 - loss: 0.2775 - val_accuracy: 0.8510 - val_auc: 0.9581 - val_loss: 0.3401
Epoch 4/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.8990 - auc: 0.9599 - loss: 0.2544 - val_accuracy: 0.8642 - val_auc: 0.9549 - val_loss: 0.2991
Epoch 5/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 124s 804ms/step - accuracy: 0.9038 - auc: 0.9647 - loss: 0.2372 - val_accuracy: 0.8858 - val_auc: 0.9584 - val_loss: 0.2819
Epoch 6/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 151s 979ms/step - accuracy: 0.8942 - auc: 0.9594 - loss: 0.2580 - val_accuracy: 0.9007 - val_auc: 0.9569 - val_loss: 0.2668
Epoch 7/50
154/154 

### Approaching overfitting 

Overfitting can be observed here as well. Given that it takes a long time to perform a full parameter search and that current performance nears CNN performance on validation set loss, I will focus on a few tries on the dropout rate and l2 regularization parameters.

In [18]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
resnet = ECGResNet(input_shape=input_shape, num_classes=num_classes, l2_reg=0.0, dropout_rate=0.3)
resnet_history = resnet.train(
    X_train, y_train, X_val, y_val, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping]
)

loss, accuracy, auc, predictions = evaluate_resnet_performance(resnet, X_val, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 173s 1s/step - accuracy: 0.7991 - auc: 0.8778 - loss: 0.4266 - val_accuracy: 0.8096 - val_auc: 0.8983 - val_loss: 0.4757
Epoch 2/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 154s 999ms/step - accuracy: 0.8679 - auc: 0.9443 - loss: 0.3002 - val_accuracy: 0.8725 - val_auc: 0.9473 - val_loss: 0.3824
Epoch 3/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 112s 729ms/step - accuracy: 0.8800 - auc: 0.9472 - loss: 0.2898 - val_accuracy: 0.8411 - val_auc: 0.9432 - val_loss: 0.3502
Epoch 4/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 153s 998ms/step - accuracy: 0.8994 - auc: 0.9588 - loss: 0.2555 - val_accuracy: 0.8841 - val_auc: 0.9592 - val_loss: 0.2714
Epoch 5/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 141s 913ms/step - accuracy: 0.8902 - auc: 0.9595 - loss: 0.2557 - val_accuracy: 0.8725 - val_auc: 0.9615 - val_loss: 0.2729
Epoch 6/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 153s 991ms/step - accuracy: 0.9109 - auc: 0.9689 - loss: 0.2273 - val_accuracy: 0.8924 - val_auc: 0.9603 - val_loss: 0.2637
Epoch 7/50
154/154 

In [19]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
resnet = ECGResNet(input_shape=input_shape, num_classes=num_classes, l2_reg=0.001, dropout_rate=0.0)
resnet_history = resnet.train(
    X_train, y_train, X_val, y_val, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping]
)

loss, accuracy, auc, predictions = evaluate_resnet_performance(resnet, X_val, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 129s 775ms/step - accuracy: 0.8050 - auc: 0.8911 - loss: 1.2017 - val_accuracy: 0.5430 - val_auc: 0.8058 - val_loss: 1.4148
Epoch 2/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 131s 851ms/step - accuracy: 0.8704 - auc: 0.9408 - loss: 0.7280 - val_accuracy: 0.6258 - val_auc: 0.8819 - val_loss: 1.0400
Epoch 3/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 146s 944ms/step - accuracy: 0.8703 - auc: 0.9411 - loss: 0.5771 - val_accuracy: 0.8560 - val_auc: 0.9533 - val_loss: 0.5784
Epoch 4/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 126s 819ms/step - accuracy: 0.8753 - auc: 0.9535 - loss: 0.4741 - val_accuracy: 0.8377 - val_auc: 0.9569 - val_loss: 0.4962
Epoch 5/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.8953 - auc: 0.9588 - loss: 0.4139 - val_accuracy: 0.9007 - val_auc: 0.9618 - val_loss: 0.3793
Epoch 6/50
154/154 ━━━━━━━━━━━━━━━━━━━━ 152s 986ms/step - accuracy: 0.9006 - auc: 0.9633 - loss: 0.3697 - val_accuracy: 0.8957 - val_auc: 0.9612 - val_loss: 0.3701
Epoch 7/50
154/154 

### Final Model 
Let's train the optimal model on all of the data.

In [20]:
X_train, y_train, X_val, y_val, X_test, y_test = create_generators_or_datasets(
    y=Y,
    window_size=None,
    step_size=None,
    batch_size=batch_size,
    sample_size=1.0,
    sampling_rate=sampling_rate,
    data_path=data_path,
    return_generators=False,
)

In [21]:
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
ecg_resnet = ECGResNet(input_shape=input_shape, num_classes=num_classes, l2_reg=0.0, dropout_rate=0.3)
resnet_history = ecg_resnet.train(
    X_train, y_train, X_val, y_val, epochs=epochs, batch_size=batch_size, callbacks=[early_stopping]
)

loss, accuracy, auc, predictions = evaluate_resnet_performance(ecg_resnet, X_val, y_val)
print(f"\nValidation set Loss: {loss:.3f}, Accuracy: {accuracy:.3f}, AUC: {auc:.3f}")

Epoch 1/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 217s 680ms/step - accuracy: 0.8228 - auc: 0.9001 - loss: 0.3905 - val_accuracy: 0.8100 - val_auc: 0.9391 - val_loss: 0.4179
Epoch 2/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 211s 680ms/step - accuracy: 0.8801 - auc: 0.9497 - loss: 0.2841 - val_accuracy: 0.8594 - val_auc: 0.9552 - val_loss: 0.3119
Epoch 3/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 214s 689ms/step - accuracy: 0.8993 - auc: 0.9608 - loss: 0.2506 - val_accuracy: 0.8745 - val_auc: 0.9572 - val_loss: 0.2934
Epoch 4/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 215s 692ms/step - accuracy: 0.8976 - auc: 0.9614 - loss: 0.2482 - val_accuracy: 0.8987 - val_auc: 0.9635 - val_loss: 0.2461
Epoch 5/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 217s 699ms/step - accuracy: 0.9077 - auc: 0.9669 - loss: 0.2303 - val_accuracy: 0.8820 - val_auc: 0.9635 - val_loss: 0.2754
Epoch 6/50
310/310 ━━━━━━━━━━━━━━━━━━━━ 219s 706ms/step - accuracy: 0.9055 - auc: 0.9662 - loss: 0.2312 - val_accuracy: 0.8962 - val_auc: 0.9659 - val_loss: 0.2518
Epoch 7/50
310/3

In [22]:
ecg_resnet.model.save("../models/resnet_model.keras", include_optimizer=False)

with open("../reports/resnet_history.pkl", "wb") as f:
    pickle.dump(resnet_history.history, f)

In [23]:
loss, accuracy, auc, predictions = evaluate_resnet_performance(ecg_resnet, X_test, y_test)
np.save("../results/resnet_preds.npy", predictions)